# 4-4: Sparse Categorical Cross Entropy

## Code.4-4-1: SCCE Calculation

In [1]:
import tensorflow as tf

from tensorflow.keras.losses import SparseCategoricalCrossentropy # one-hot encoding이 아닐때 사용(SCCE)

batch_size, n_class = 16, 5

predictions = tf.random.uniform(shape=(batch_size, n_class), minval=0, maxval=1, dtype=tf.float32)

pred_sum = tf.reshape(tf.reduce_sum(predictions, axis=1), (-1, 1)) # tf.reduce_sum(predictions, axis=1) row(행) 기준 다 더하면 1이나와야함(확률이니까)
#print(predictions.shape, pred_sum.shape)  # batch, class 크기로 만들어지고, reshape -1,1 은 16 x 1 로 만들어짐 (-1 전체, 1 열)

predictions = predictions / pred_sum

labels = tf.random.uniform(shape=(batch_size, ), minval=0, maxval=n_class, dtype=tf.int32)  # shape=(batch_size, )인 이유는 백터로 만들어 놨기 때문이다.
# SCCE는 데이터 lable이 클래스 인덱스를 가지는 벡터일때 사용한다.

# print(labels) # tf.Tensor([4 3 1 4 4 4 2 2 0 0 3 4 0 0 0 3], shape=(16,), dtype=int32) # 4는 5번째 클래스

loss_object = SparseCategoricalCrossentropy()
loss = loss_object(labels, predictions)

print(loss.numpy())

ce = 0
for label, prediction in zip(labels, predictions):
  ce += -tf.math.log(prediction[label])

ce /= batch_size
print(ce.numpy())

1.8034806
1.8034806


## Code.4-4-2: SCCE with Model/Dataset

In [9]:
# 4-1에 있던 내용
from numpy.core.fromnumeric import shape
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

N, n_feature = 100, 2
n_class = 5

X = tf.zeros(shape=(0, n_feature))
Y = tf.zeros(shape=(0,1), dtype=tf.int32)

for class_idx in range(n_class):
  center = tf.random.uniform(minval=-15, maxval=15, shape=(2,)) # x, y, 좌표 생성

  x1 = center[0] + tf.random.normal(shape=(N, 1)) # x 축
  x2 = center[1] + tf.random.normal(shape=(N, 1)) # y 축

  #ax.scatter(x1.numpy(), x2.numpy())  # N * n_class 로 각 class당 150개가 찍힌다.  
  #지금은 N, n_class 숫자를 보기 편하게 바꿨음

  x = tf.concat((x1, x2), axis=1) # column 백터 2개가 있으니까 가로축으로 합치기?
  y = class_idx*tf.ones(shape=(N, 1), dtype=tf.int32)
  #print(x.shape, y.shape)

  X = tf.concat((X, x), axis=0) # 세로축으로 해야 데이터들이 쌓임
  Y = tf.concat((Y, y), axis=0)
  #print(X.shape, Y.shape)

dataset = tf.data.Dataset.from_tensor_slices((X, Y))  # X, Y 튜플로 넣어줌
dataset = dataset.batch(batch_size)

model = Dense(units=n_class, activation='softmax')
loss_object = SparseCategoricalCrossentropy()

for x, y in dataset:
  predictions = model(x)
  #print(predictions.shape, y.shape) #(16, 5) (16, 1) ... (4, 5) (4, 1)
  loss = loss_object(y, predictions)
  print(loss.numpy())

3.3052335
3.2585597
3.2584763
3.2832332
3.2406175
3.2059393
7.1595163
8.570461
8.752533
8.934972
8.601059
8.689228
6.045042
3.7976573
3.5127952
3.6947956
3.5994802
3.5396357
2.9581714
0.90106875
0.881211
0.8950073
0.8966636
0.96953976
0.8404144
23.17139
23.590939
23.944168
23.557598
23.217945
23.163832
24.549566


# 4-5: Categorical Cross Entropy


## Code.4-5-1: CCE Calculation

In [17]:
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy

batch_size, n_class = 16, 5

predictions = tf.random.uniform(shape=(batch_size, n_class), minval=0, maxval=1, dtype=tf.float32)

pred_sum = tf.reshape(tf.reduce_sum(predictions, axis=1), (-1, 1)) 
predictions = predictions / pred_sum

labels = tf.random.uniform(shape=(batch_size, ), minval=0, maxval=n_class, dtype=tf.int32)
#print(labels) #tf.Tensor([4 3 1 2 4 1 4 4 2 2 3 1 0 1 3 0], shape=(16,), dtype=int32)

labels = tf.one_hot(labels, n_class)
#print(labels) # [[0 0 0 0 1] [1 0 0 0 0] ... [1 0 0 0 0]], shape=(16,5), dtype=float32

loss_object = CategoricalCrossentropy() # Categorical은 y labels이 one-hot encoding이 되어있을 때 사용
loss = loss_object(labels, predictions)

print("CCE(Tensorflow): ", loss.numpy())

cce_man = tf.reduce_mean(tf.reduce_sum(-labels*tf.math.log(predictions), axis=1))
print("CCE(Manual): ", cce_man.numpy())

CCE(Tensorflow):  2.2558763
CCE(Manual):  2.2558763


## Code.4-5-2: CCE with Model/Dataset

In [20]:
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy

N, n_feature = 8, 2
n_class = 5

X = tf.zeros(shape=(0, n_feature))
Y = tf.zeros(shape=(0, ), dtype=tf.int32)

for class_idx in range(n_class):
  center = tf.random.uniform(minval=-15, maxval=15, shape=(2,))

  x1 = center[0] + tf.random.normal(shape=(N, 1))
  x2 = center[1] + tf.random.normal(shape=(N, 1))

  x = tf.concat((x1, x2), axis=1)
  y = class_idx*tf.ones(shape=(N, ), dtype=tf.int32)  # 위 zeros코드, 여기코드를 백터형식으로 바꿨음  -> 즉 일차배열 벡터형태로 만듦

  X = tf.concat((X, x), axis=0) 
  Y = tf.concat((Y, y), axis=0)

Y = tf.one_hot(Y, depth=n_class, dtype=tf.int32)  # one-hot encoding

dataset = tf.data.Dataset.from_tensor_slices((X, Y))  # X, Y 튜플로 넣어줌
dataset = dataset.batch(batch_size)

model = Dense(units=n_class, activation='softmax')
loss_object = CategoricalCrossentropy()

for x, y in dataset:
  predictions = model(x)
  #print(predictions.shape, y.shape)
  loss = loss_object(y, predictions)
  print(loss.numpy())

5.009859
8.037728
0.029731346
